# CSDI: Conditional Score-based Diffusion для Imputation и Forecasting

CSDI (Conditional Score-based Diffusion for Imputation) - это non-autoregressive модель, которая может выполнять:

1. **Imputation (заполнение пропусков)**: Восстановление недостающих значений во временном ряду
2. **Forecasting (прогнозирование)**: Генерация будущих значений как "заполнение" будущих timesteps
3. **Interpolation**: Восстановление значений между известными точками

## Ключевые преимущества CSDI:

- **Non-autoregressive**: Генерирует все значения одновременно (нет накопления ошибок)
- **Conditional**: Учитывает известные значения как условие
- **Гибкость**: Работает с произвольными паттернами пропусков

**Статья**: Tashiro et al., "CSDI: Conditional Score-based Diffusion Models for Probabilistic Time Series Imputation" (NeurIPS 2021)

In [ ]:
# Установка зависимостей
!pip install torch numpy pandas matplotlib seaborn scikit-learn tqdm einops

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, Optional, Dict, List
from dataclasses import dataclass
from tqdm import tqdm
from einops import rearrange, repeat
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 1. Генерация синтетических данных с пропусками

In [ ]:
def generate_crypto_like_data(
    n_samples: int = 1000,
    n_features: int = 5,
    seq_len: int = 100
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Генерация синтетических данных, похожих на криптовалютные.
    
    Returns:
        data: [n_samples, seq_len, n_features]
        masks: [n_samples, seq_len, n_features] - 1 где данные есть, 0 где пропуски
    """
    np.random.seed(42)
    
    # Базовый тренд (GBM)
    dt = 1/24  # почасовые данные
    mu = 0.0001
    sigma = 0.02
    
    data = np.zeros((n_samples, seq_len, n_features))
    
    for i in range(n_samples):
        # Close price (feature 0)
        returns = np.random.normal(mu * dt, sigma * np.sqrt(dt), seq_len)
        prices = 100 * np.exp(np.cumsum(returns))
        data[i, :, 0] = prices
        
        # Volume (feature 1) - зависит от волатильности
        vol = np.abs(returns) * 100
        data[i, :, 1] = np.random.exponential(1 + vol) * 1000
        
        # RSI-like (feature 2)
        gains = np.maximum(returns, 0)
        losses = np.maximum(-returns, 0)
        avg_gain = pd.Series(gains).rolling(14, min_periods=1).mean().values
        avg_loss = pd.Series(losses).rolling(14, min_periods=1).mean().values
        rs = avg_gain / (avg_loss + 1e-10)
        data[i, :, 2] = 100 - 100 / (1 + rs)
        
        # Volatility (feature 3)
        data[i, :, 3] = pd.Series(returns).rolling(24, min_periods=1).std().values * np.sqrt(24) * 100
        
        # Momentum (feature 4)
        data[i, :, 4] = pd.Series(prices).pct_change(12).fillna(0).values * 100
    
    # Нормализация
    for f in range(n_features):
        mean = data[:, :, f].mean()
        std = data[:, :, f].std()
        data[:, :, f] = (data[:, :, f] - mean) / (std + 1e-10)
    
    # Создаём маски пропусков
    masks = np.ones_like(data)
    
    for i in range(n_samples):
        # Случайные пропуски (10-30% данных)
        missing_rate = np.random.uniform(0.1, 0.3)
        n_missing = int(seq_len * n_features * missing_rate)
        
        # Случайные позиции для пропусков
        flat_indices = np.random.choice(
            seq_len * n_features, 
            size=n_missing, 
            replace=False
        )
        
        for idx in flat_indices:
            t = idx // n_features
            f = idx % n_features
            masks[i, t, f] = 0
    
    return data, masks

# Генерируем данные
data, masks = generate_crypto_like_data(n_samples=500, n_features=5, seq_len=100)

print(f"Data shape: {data.shape}")
print(f"Masks shape: {masks.shape}")
print(f"Средняя доля пропусков: {1 - masks.mean():.2%}")

In [ ]:
# Визуализация пропусков
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

sample_idx = 0

# Маска пропусков
ax1 = axes[0, 0]
im = ax1.imshow(masks[sample_idx].T, aspect='auto', cmap='RdYlGn', vmin=0, vmax=1)
ax1.set_xlabel('Time')
ax1.set_ylabel('Feature')
ax1.set_title('Missing Data Mask (green=observed, red=missing)')
ax1.set_yticks(range(5))
ax1.set_yticklabels(['Close', 'Volume', 'RSI', 'Volatility', 'Momentum'])
plt.colorbar(im, ax=ax1)

# Данные с пропусками (Close price)
ax2 = axes[0, 1]
observed_data = data[sample_idx, :, 0].copy()
observed_data[masks[sample_idx, :, 0] == 0] = np.nan
ax2.plot(observed_data, 'b.-', markersize=2)
ax2.set_xlabel('Time')
ax2.set_ylabel('Normalized Price')
ax2.set_title('Close Price with Missing Values')

# Статистика пропусков по фичам
ax3 = axes[1, 0]
missing_by_feature = 1 - masks.mean(axis=(0, 1))
features = ['Close', 'Volume', 'RSI', 'Volatility', 'Momentum']
ax3.bar(features, missing_by_feature)
ax3.set_ylabel('Missing Rate')
ax3.set_title('Missing Rate by Feature')

# Статистика пропусков по времени
ax4 = axes[1, 1]
missing_by_time = 1 - masks.mean(axis=(0, 2))
ax4.plot(missing_by_time)
ax4.set_xlabel('Time')
ax4.set_ylabel('Missing Rate')
ax4.set_title('Missing Rate Over Time')

plt.tight_layout()
plt.show()

## 2. CSDI Architecture

CSDI использует score-based подход с кондиционированием на наблюдаемые значения.

Основные компоненты:
1. **Diffusion Embedding**: Кодирование шага диффузии
2. **Feature Embedding**: Кодирование признаков
3. **Bidirectional Transformer**: Обработка последовательности
4. **Conditional Mechanism**: Учёт известных значений

In [ ]:
@dataclass
class CSDIConfig:
    """Конфигурация CSDI."""
    seq_len: int = 100
    n_features: int = 5
    hidden_dim: int = 64
    num_heads: int = 4
    num_layers: int = 4
    diffusion_steps: int = 50
    beta_start: float = 0.0001
    beta_end: float = 0.5
    schedule: str = 'quad'  # 'linear', 'quad', 'cosine'

config = CSDIConfig()
print(f"Config: {config}")

In [ ]:
class DiffusionEmbedding(nn.Module):
    """Кодирование шага диффузии."""
    
    def __init__(self, dim: int, max_steps: int = 500):
        super().__init__()
        self.dim = dim
        self.max_steps = max_steps
        
        # Предвычисляем embeddings
        self.register_buffer(
            'embedding', 
            self._build_embedding(max_steps, dim)
        )
        
        self.projection = nn.Sequential(
            nn.Linear(dim, dim * 2),
            nn.SiLU(),
            nn.Linear(dim * 2, dim)
        )
    
    def _build_embedding(self, max_steps: int, dim: int) -> torch.Tensor:
        """Синусоидальное кодирование."""
        steps = torch.arange(max_steps).unsqueeze(1)
        frequencies = 10000 ** (-torch.arange(0, dim, 2).float() / dim)
        
        encoding = torch.zeros(max_steps, dim)
        encoding[:, 0::2] = torch.sin(steps * frequencies)
        encoding[:, 1::2] = torch.cos(steps * frequencies)
        
        return encoding
    
    def forward(self, t: torch.Tensor) -> torch.Tensor:
        """
        Args:
            t: [batch] - шаги времени
        Returns:
            [batch, dim]
        """
        t = t.long().clamp(0, self.max_steps - 1)
        emb = self.embedding[t]
        return self.projection(emb)

In [ ]:
class PositionalEncoding(nn.Module):
    """Позиционное кодирование для временного ряда."""
    
    def __init__(self, dim: int, max_len: int = 500):
        super().__init__()
        
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, dim, 2).float() * -(np.log(10000.0) / dim))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: [batch, seq_len, dim]
        """
        return x + self.pe[:, :x.size(1)]

In [ ]:
class ConditionalResidualBlock(nn.Module):
    """Residual block с кондиционированием на время диффузии."""
    
    def __init__(self, hidden_dim: int, diffusion_dim: int):
        super().__init__()
        
        self.norm1 = nn.LayerNorm(hidden_dim)
        self.linear1 = nn.Linear(hidden_dim, hidden_dim * 2)
        self.norm2 = nn.LayerNorm(hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        
        # Projection для условия времени
        self.cond_proj = nn.Linear(diffusion_dim, hidden_dim * 2)
    
    def forward(
        self, 
        x: torch.Tensor, 
        cond: torch.Tensor
    ) -> torch.Tensor:
        """
        Args:
            x: [batch, seq_len, hidden_dim]
            cond: [batch, diffusion_dim]
        """
        # Условие → scale и shift
        cond = self.cond_proj(cond)  # [batch, hidden_dim * 2]
        scale, shift = cond.chunk(2, dim=-1)  # каждый [batch, hidden_dim]
        
        # Первый слой с AdaIN
        h = self.norm1(x)
        h = h * (1 + scale.unsqueeze(1)) + shift.unsqueeze(1)
        h = self.linear1(h)
        h = F.gelu(h)
        h = h[..., :h.size(-1)//2] * torch.sigmoid(h[..., h.size(-1)//2:])  # GLU
        
        # Второй слой
        h = self.norm2(h)
        h = self.linear2(h)
        
        return x + h

In [ ]:
class TransformerBlock(nn.Module):
    """Transformer block с кондиционированием."""
    
    def __init__(
        self, 
        hidden_dim: int, 
        num_heads: int, 
        diffusion_dim: int,
        dropout: float = 0.1
    ):
        super().__init__()
        
        # Self-attention
        self.norm1 = nn.LayerNorm(hidden_dim)
        self.attn = nn.MultiheadAttention(
            hidden_dim, 
            num_heads, 
            dropout=dropout,
            batch_first=True
        )
        
        # Feedforward с кондиционированием
        self.res_block = ConditionalResidualBlock(hidden_dim, diffusion_dim)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(
        self, 
        x: torch.Tensor, 
        cond: torch.Tensor,
        mask: Optional[torch.Tensor] = None
    ) -> torch.Tensor:
        """
        Args:
            x: [batch, seq_len, hidden_dim]
            cond: [batch, diffusion_dim]
            mask: [batch, seq_len] - attention mask
        """
        # Self-attention
        h = self.norm1(x)
        h, _ = self.attn(h, h, h, key_padding_mask=mask)
        x = x + self.dropout(h)
        
        # Conditional feedforward
        x = self.res_block(x, cond)
        
        return x

In [ ]:
class CSDI(nn.Module):
    """CSDI: Conditional Score-based Diffusion for Imputation."""
    
    def __init__(self, config: CSDIConfig):
        super().__init__()
        
        self.config = config
        
        # Input projection
        # Вход: (значение, маска) для каждой фичи
        self.input_proj = nn.Linear(config.n_features * 2, config.hidden_dim)
        
        # Diffusion embedding
        self.diff_embed = DiffusionEmbedding(config.hidden_dim, config.diffusion_steps)
        
        # Positional encoding
        self.pos_enc = PositionalEncoding(config.hidden_dim, config.seq_len)
        
        # Transformer layers
        self.layers = nn.ModuleList([
            TransformerBlock(
                config.hidden_dim, 
                config.num_heads, 
                config.hidden_dim
            )
            for _ in range(config.num_layers)
        ])
        
        # Output projection
        self.output_proj = nn.Sequential(
            nn.LayerNorm(config.hidden_dim),
            nn.Linear(config.hidden_dim, config.hidden_dim),
            nn.GELU(),
            nn.Linear(config.hidden_dim, config.n_features)
        )
        
        # Noise schedule
        self._setup_noise_schedule()
    
    def _setup_noise_schedule(self):
        """Настройка noise schedule."""
        T = self.config.diffusion_steps
        
        if self.config.schedule == 'linear':
            betas = torch.linspace(self.config.beta_start, self.config.beta_end, T)
        elif self.config.schedule == 'quad':
            betas = torch.linspace(self.config.beta_start**0.5, self.config.beta_end**0.5, T) ** 2
        elif self.config.schedule == 'cosine':
            steps = torch.arange(T + 1) / T
            alphas_bar = torch.cos((steps + 0.008) / 1.008 * np.pi / 2) ** 2
            betas = 1 - alphas_bar[1:] / alphas_bar[:-1]
            betas = betas.clamp(max=0.999)
        else:
            raise ValueError(f"Unknown schedule: {self.config.schedule}")
        
        alphas = 1 - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        
        self.register_buffer('betas', betas)
        self.register_buffer('alphas', alphas)
        self.register_buffer('alphas_cumprod', alphas_cumprod)
        self.register_buffer('sqrt_alphas_cumprod', torch.sqrt(alphas_cumprod))
        self.register_buffer('sqrt_one_minus_alphas_cumprod', torch.sqrt(1 - alphas_cumprod))
        self.register_buffer('sqrt_recip_alphas', torch.sqrt(1 / alphas))
    
    def forward_diffusion(
        self,
        x0: torch.Tensor,
        t: torch.Tensor,
        noise: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Прямой диффузионный процесс.
        
        Args:
            x0: [batch, seq_len, n_features] - чистые данные
            t: [batch] - шаги времени
        Returns:
            xt, noise
        """
        if noise is None:
            noise = torch.randn_like(x0)
        
        sqrt_alpha = self.sqrt_alphas_cumprod[t].view(-1, 1, 1)
        sqrt_one_minus_alpha = self.sqrt_one_minus_alphas_cumprod[t].view(-1, 1, 1)
        
        xt = sqrt_alpha * x0 + sqrt_one_minus_alpha * noise
        
        return xt, noise
    
    def denoise(
        self,
        xt: torch.Tensor,
        t: torch.Tensor,
        observed: torch.Tensor,
        mask: torch.Tensor
    ) -> torch.Tensor:
        """
        Предсказание шума.
        
        Args:
            xt: [batch, seq_len, n_features] - зашумлённые данные
            t: [batch] - шаги диффузии
            observed: [batch, seq_len, n_features] - наблюдаемые значения
            mask: [batch, seq_len, n_features] - маска (1=observed, 0=missing)
        """
        batch_size = xt.shape[0]
        
        # Объединяем условие и зашумлённые данные
        # Для наблюдаемых точек используем observed, для пропусков - xt
        conditioned = observed * mask + xt * (1 - mask)
        
        # Конкатенируем значения и маску
        x_input = torch.cat([conditioned, mask], dim=-1)  # [batch, seq_len, n_features*2]
        
        # Input projection
        h = self.input_proj(x_input)  # [batch, seq_len, hidden_dim]
        
        # Positional encoding
        h = self.pos_enc(h)
        
        # Diffusion embedding
        diff_emb = self.diff_embed(t.float())  # [batch, hidden_dim]
        
        # Transformer layers
        for layer in self.layers:
            h = layer(h, diff_emb)
        
        # Output
        noise_pred = self.output_proj(h)  # [batch, seq_len, n_features]
        
        return noise_pred
    
    def compute_loss(
        self,
        x: torch.Tensor,
        mask: torch.Tensor
    ) -> torch.Tensor:
        """
        Вычислить loss для обучения.
        
        Args:
            x: [batch, seq_len, n_features] - полные данные
            mask: [batch, seq_len, n_features] - маска наблюдаемых
        """
        batch_size = x.shape[0]
        device = x.device
        
        # Случайный шаг диффузии
        t = torch.randint(0, self.config.diffusion_steps, (batch_size,), device=device)
        
        # Прямая диффузия
        xt, noise = self.forward_diffusion(x, t)
        
        # Предсказание шума
        # Наблюдаемые данные - это x * mask (значения там, где есть данные)
        noise_pred = self.denoise(xt, t, x * mask, mask)
        
        # Loss только на пропущенных значениях
        target_mask = 1 - mask  # инвертируем - хотим предсказывать пропуски
        
        loss = F.mse_loss(noise_pred * target_mask, noise * target_mask, reduction='sum')
        loss = loss / (target_mask.sum() + 1e-10)
        
        return loss
    
    @torch.no_grad()
    def sample(
        self,
        observed: torch.Tensor,
        mask: torch.Tensor,
        num_samples: int = 1
    ) -> torch.Tensor:
        """
        Генерация сэмплов (imputation).
        
        Args:
            observed: [batch, seq_len, n_features] - наблюдаемые данные
            mask: [batch, seq_len, n_features] - маска
            num_samples: количество сэмплов
            
        Returns:
            samples: [batch, num_samples, seq_len, n_features]
        """
        batch_size, seq_len, n_features = observed.shape
        device = observed.device
        
        all_samples = []
        
        for _ in range(num_samples):
            # Начинаем с шума для пропущенных позиций
            x = torch.randn(batch_size, seq_len, n_features, device=device)
            
            # Наблюдаемые значения остаются фиксированными
            x = observed * mask + x * (1 - mask)
            
            # Обратный процесс
            for t in reversed(range(self.config.diffusion_steps)):
                t_tensor = torch.full((batch_size,), t, device=device)
                
                # Предсказываем шум
                noise_pred = self.denoise(x, t_tensor, observed, mask)
                
                # DDPM update (только для пропусков)
                alpha = self.alphas[t]
                alpha_cumprod = self.alphas_cumprod[t]
                beta = self.betas[t]
                
                # x_{t-1} = 1/sqrt(alpha) * (x_t - beta/sqrt(1-alpha_bar) * noise_pred) + sigma * z
                coef1 = 1 / torch.sqrt(alpha)
                coef2 = beta / torch.sqrt(1 - alpha_cumprod)
                
                mean = coef1 * (x - coef2 * noise_pred)
                
                if t > 0:
                    # Posterior variance
                    alpha_cumprod_prev = self.alphas_cumprod[t - 1]
                    variance = beta * (1 - alpha_cumprod_prev) / (1 - alpha_cumprod)
                    noise = torch.randn_like(x)
                    x = mean + torch.sqrt(variance) * noise
                else:
                    x = mean
                
                # Сохраняем наблюдаемые значения
                x = observed * mask + x * (1 - mask)
            
            all_samples.append(x)
        
        return torch.stack(all_samples, dim=1)  # [batch, num_samples, seq_len, n_features]

# Создаём модель
model = CSDI(config).to(device)
print(f"Параметров: {sum(p.numel() for p in model.parameters()):,}")

## 3. Подготовка данных и обучение

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ImputationDataset(Dataset):
    def __init__(self, data: np.ndarray, masks: np.ndarray):
        self.data = torch.FloatTensor(data)
        self.masks = torch.FloatTensor(masks)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.masks[idx]

# Train/val/test split
n = len(data)
train_size = int(0.7 * n)
val_size = int(0.15 * n)

train_data, train_masks = data[:train_size], masks[:train_size]
val_data, val_masks = data[train_size:train_size+val_size], masks[train_size:train_size+val_size]
test_data, test_masks = data[train_size+val_size:], masks[train_size+val_size:]

train_dataset = ImputationDataset(train_data, train_masks)
val_dataset = ImputationDataset(val_data, val_masks)
test_dataset = ImputationDataset(test_data, test_masks)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

In [ ]:
def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    
    for data, mask in loader:
        data = data.to(device)
        mask = mask.to(device)
        
        optimizer.zero_grad()
        loss = model.compute_loss(data, mask)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)


def validate(model, loader, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for data, mask in loader:
            data = data.to(device)
            mask = mask.to(device)
            loss = model.compute_loss(data, mask)
            total_loss += loss.item()
    
    return total_loss / len(loader)

In [ ]:
# Обучение
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

n_epochs = 30
train_losses = []
val_losses = []
best_val_loss = float('inf')

for epoch in tqdm(range(n_epochs), desc="Training"):
    train_loss = train_epoch(model, train_loader, optimizer, device)
    val_loss = validate(model, val_loader, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    scheduler.step()
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_state = model.state_dict().copy()
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}: Train={train_loss:.4f}, Val={val_loss:.4f}")

model.load_state_dict(best_state)
print(f"\nBest Val Loss: {best_val_loss:.4f}")

In [ ]:
# График обучения
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('CSDI Training Progress')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Imputation Results

In [ ]:
def evaluate_imputation(
    model: CSDI,
    data: np.ndarray,
    masks: np.ndarray,
    num_samples: int = 10
) -> Dict[str, float]:
    """
    Оценка качества imputation.
    """
    model.eval()
    
    data_tensor = torch.FloatTensor(data).to(device)
    mask_tensor = torch.FloatTensor(masks).to(device)
    
    # Наблюдаемые данные
    observed = data_tensor * mask_tensor
    
    # Генерируем сэмплы
    with torch.no_grad():
        samples = model.sample(observed, mask_tensor, num_samples)
    
    # Среднее предсказание
    mean_pred = samples.mean(dim=1)  # [batch, seq_len, n_features]
    
    # Метрики только на пропущенных значениях
    missing_mask = 1 - mask_tensor
    
    # MSE
    mse = ((mean_pred - data_tensor) ** 2 * missing_mask).sum() / missing_mask.sum()
    rmse = torch.sqrt(mse)
    
    # MAE
    mae = (torch.abs(mean_pred - data_tensor) * missing_mask).sum() / missing_mask.sum()
    
    # CRPS (simplified)
    samples_np = samples.cpu().numpy()
    data_np = data_tensor.cpu().numpy()
    missing_np = missing_mask.cpu().numpy()
    
    # Approximate CRPS
    crps = np.mean(np.abs(samples_np - data_np[:, np.newaxis, :, :]) * missing_np[:, np.newaxis, :, :])
    
    return {
        'RMSE': rmse.item(),
        'MAE': mae.item(),
        'CRPS': crps,
        'samples': samples.cpu().numpy(),
        'mean_pred': mean_pred.cpu().numpy()
    }

# Оценка на тестовых данных
results = evaluate_imputation(model, test_data, test_masks, num_samples=10)

print("=== CSDI Imputation Results ===")
print(f"RMSE: {results['RMSE']:.4f}")
print(f"MAE: {results['MAE']:.4f}")
print(f"CRPS: {results['CRPS']:.4f}")

In [ ]:
# Визуализация imputation
fig, axes = plt.subplots(3, 2, figsize=(14, 10))

for i, ax_row in enumerate(axes):
    sample_idx = i
    feature_idx = 0  # Close price
    
    # Левый график: Close price
    ax = ax_row[0]
    
    # Истинные значения
    true_values = test_data[sample_idx, :, feature_idx]
    
    # Наблюдаемые
    observed = true_values.copy()
    observed[test_masks[sample_idx, :, feature_idx] == 0] = np.nan
    
    # Предсказанные
    pred_mean = results['mean_pred'][sample_idx, :, feature_idx]
    samples = results['samples'][sample_idx, :, :, feature_idx]  # [num_samples, seq_len]
    pred_std = samples.std(axis=0)
    
    time = np.arange(len(true_values))
    
    # CI для пропусков
    missing_mask = test_masks[sample_idx, :, feature_idx] == 0
    ax.fill_between(
        time[missing_mask],
        pred_mean[missing_mask] - 2 * pred_std[missing_mask],
        pred_mean[missing_mask] + 2 * pred_std[missing_mask],
        alpha=0.3,
        color='orange',
        label='95% CI'
    )
    
    ax.plot(time, true_values, 'g--', alpha=0.5, label='True (hidden)')
    ax.scatter(time[~missing_mask], observed[~missing_mask], c='blue', s=10, label='Observed')
    ax.plot(time[missing_mask], pred_mean[missing_mask], 'ro', markersize=3, label='Imputed')
    
    ax.set_title(f'Sample {i+1}: Close Price')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    if i == 0:
        ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Правый график: другая фича
    ax = ax_row[1]
    feature_idx = 2  # RSI
    
    true_values = test_data[sample_idx, :, feature_idx]
    observed = true_values.copy()
    observed[test_masks[sample_idx, :, feature_idx] == 0] = np.nan
    pred_mean = results['mean_pred'][sample_idx, :, feature_idx]
    
    missing_mask = test_masks[sample_idx, :, feature_idx] == 0
    
    ax.plot(time, true_values, 'g--', alpha=0.5, label='True')
    ax.scatter(time[~missing_mask], observed[~missing_mask], c='blue', s=10, label='Observed')
    ax.plot(time[missing_mask], pred_mean[missing_mask], 'ro', markersize=3, label='Imputed')
    
    ax.set_title(f'Sample {i+1}: RSI')
    ax.set_xlabel('Time')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. CSDI для Forecasting

Для прогнозирования мы просто маскируем будущие timesteps и используем imputation для их заполнения.

In [ ]:
def forecast_with_csdi(
    model: CSDI,
    history: np.ndarray,
    forecast_horizon: int,
    num_samples: int = 50
) -> Dict[str, np.ndarray]:
    """
    Прогнозирование с помощью CSDI.
    
    Args:
        model: обученная CSDI модель
        history: [seq_len, n_features] - исторические данные
        forecast_horizon: горизонт прогноза
        num_samples: количество сэмплов
    """
    model.eval()
    
    seq_len, n_features = history.shape
    total_len = seq_len + forecast_horizon
    
    # Дополняем нулями для будущих значений
    padded_data = np.zeros((total_len, n_features))
    padded_data[:seq_len] = history
    
    # Маска: 1 для истории, 0 для прогноза
    mask = np.zeros((total_len, n_features))
    mask[:seq_len] = 1
    
    # Если модель обучена на фиксированной длине, обрезаем или дополняем
    if total_len != model.config.seq_len:
        # Используем последние seq_len точек истории + прогноз
        if total_len > model.config.seq_len:
            start_idx = total_len - model.config.seq_len
            padded_data = padded_data[start_idx:]
            mask = mask[start_idx:]
        else:
            # Дополняем нулями в начале
            pad_len = model.config.seq_len - total_len
            padded_data = np.pad(padded_data, ((pad_len, 0), (0, 0)))
            mask = np.pad(mask, ((pad_len, 0), (0, 0)))
    
    # Конвертируем в тензоры
    data_tensor = torch.FloatTensor(padded_data).unsqueeze(0).to(device)
    mask_tensor = torch.FloatTensor(mask).unsqueeze(0).to(device)
    
    observed = data_tensor * mask_tensor
    
    # Генерируем сэмплы
    with torch.no_grad():
        samples = model.sample(observed, mask_tensor, num_samples)
    
    samples_np = samples.cpu().numpy()[0]  # [num_samples, seq_len, n_features]
    
    # Извлекаем прогноз (последние forecast_horizon точек)
    forecast_samples = samples_np[:, -forecast_horizon:, :]
    
    return {
        'samples': forecast_samples,
        'mean': forecast_samples.mean(axis=0),
        'std': forecast_samples.std(axis=0),
        'q10': np.percentile(forecast_samples, 10, axis=0),
        'q50': np.percentile(forecast_samples, 50, axis=0),
        'q90': np.percentile(forecast_samples, 90, axis=0),
    }

In [ ]:
# Тест прогнозирования
# Берём историю из тестовых данных
test_sample = test_data[0]
history_len = 80
forecast_horizon = 20

history = test_sample[:history_len]
actual_future = test_sample[history_len:history_len + forecast_horizon]

forecast = forecast_with_csdi(model, history, forecast_horizon, num_samples=50)

print(f"Forecast shape: {forecast['mean'].shape}")

In [ ]:
# Визуализация прогноза
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

features = ['Close', 'Volume', 'RSI', 'Volatility']

for idx, (ax, feature_name) in enumerate(zip(axes.flatten(), features)):
    # История
    time_history = np.arange(history_len)
    time_forecast = np.arange(history_len, history_len + forecast_horizon)
    
    ax.plot(time_history, history[:, idx], 'b-', label='History')
    
    # Прогноз
    ax.fill_between(
        time_forecast,
        forecast['q10'][:, idx],
        forecast['q90'][:, idx],
        alpha=0.3,
        color='orange',
        label='80% CI'
    )
    ax.plot(time_forecast, forecast['mean'][:, idx], 'orange', linewidth=2, label='Forecast')
    
    # Истинные значения (если есть)
    ax.plot(time_forecast, actual_future[:, idx], 'g--', linewidth=2, label='Actual')
    
    ax.axvline(history_len, color='gray', linestyle='--', alpha=0.5)
    ax.set_title(f'{feature_name}')
    ax.set_xlabel('Time')
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.suptitle('CSDI Forecasting Results', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Сравнение с базовыми методами

In [ ]:
class MeanImputer:
    """Заполнение пропусков средним значением."""
    
    def impute(self, data: np.ndarray, mask: np.ndarray) -> np.ndarray:
        result = data.copy()
        for f in range(data.shape[-1]):
            feature_data = data[..., f]
            feature_mask = mask[..., f]
            mean_val = np.mean(feature_data[feature_mask == 1])
            result[..., f] = np.where(feature_mask == 1, feature_data, mean_val)
        return result


class LinearInterpolator:
    """Линейная интерполяция."""
    
    def impute(self, data: np.ndarray, mask: np.ndarray) -> np.ndarray:
        result = data.copy()
        for i in range(data.shape[0]):  # для каждого сэмпла
            for f in range(data.shape[-1]):  # для каждой фичи
                series = pd.Series(data[i, :, f])
                series[mask[i, :, f] == 0] = np.nan
                result[i, :, f] = series.interpolate(method='linear').fillna(method='bfill').fillna(method='ffill').values
        return result


# Сравнение методов
mean_imputer = MeanImputer()
linear_imputer = LinearInterpolator()

mean_imputed = mean_imputer.impute(test_data, test_masks)
linear_imputed = linear_imputer.impute(test_data, test_masks)

# Метрики
missing_mask = 1 - test_masks

def compute_rmse(pred, true, mask):
    return np.sqrt(np.sum((pred - true) ** 2 * mask) / np.sum(mask))

def compute_mae(pred, true, mask):
    return np.sum(np.abs(pred - true) * mask) / np.sum(mask)

print("=== Сравнение методов Imputation ===")
print(f"\n{'Method':<20} {'RMSE':<10} {'MAE':<10}")
print("-" * 40)

print(f"{'Mean Imputation':<20} {compute_rmse(mean_imputed, test_data, missing_mask):<10.4f} {compute_mae(mean_imputed, test_data, missing_mask):<10.4f}")
print(f"{'Linear Interpolation':<20} {compute_rmse(linear_imputed, test_data, missing_mask):<10.4f} {compute_mae(linear_imputed, test_data, missing_mask):<10.4f}")
print(f"{'CSDI':<20} {results['RMSE']:<10.4f} {results['MAE']:<10.4f}")

In [ ]:
# Визуализация сравнения
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

sample_idx = 0
feature_idx = 0

time = np.arange(100)
true_values = test_data[sample_idx, :, feature_idx]
mask = test_masks[sample_idx, :, feature_idx]

for ax, (method_name, imputed) in zip(axes, [
    ('Mean Imputation', mean_imputed[sample_idx, :, feature_idx]),
    ('Linear Interpolation', linear_imputed[sample_idx, :, feature_idx]),
    ('CSDI', results['mean_pred'][sample_idx, :, feature_idx])
]):
    ax.plot(time, true_values, 'g--', alpha=0.5, label='True')
    ax.scatter(time[mask == 1], true_values[mask == 1], c='blue', s=10, label='Observed')
    ax.scatter(time[mask == 0], imputed[mask == 0], c='red', s=15, marker='x', label='Imputed')
    
    # Ошибка
    rmse = np.sqrt(np.mean((imputed[mask == 0] - true_values[mask == 0]) ** 2))
    
    ax.set_title(f'{method_name}\nRMSE: {rmse:.4f}')
    ax.set_xlabel('Time')
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Выводы

### Преимущества CSDI:

1. **Универсальность**: Работает как для imputation, так и для forecasting
2. **Вероятностные предсказания**: Генерирует распределение возможных значений
3. **Non-autoregressive**: Нет накопления ошибок
4. **Гибкость**: Работает с произвольными паттернами пропусков

### Ограничения:

1. **Вычислительная сложность**: Требует много памяти для длинных последовательностей
2. **Фиксированная длина**: Модель обучена на фиксированной seq_len
3. **Медленный sampling**: DDPM sampling требует много шагов

### Рекомендации:

- Используйте DDIM sampling для ускорения генерации
- Для forecasting рассмотрите комбинацию с TimeGrad
- Для очень длинных последовательностей рассмотрите Diffusion-TS

In [ ]:
# Сохранение модели
torch.save({
    'model_state_dict': model.state_dict(),
    'config': config,
    'train_losses': train_losses,
    'val_losses': val_losses,
}, 'csdi_model.pt')

print("Model saved to csdi_model.pt")